---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    import numpy as np
    import pandas as pd
    '''Returns a DataFrame of towns and the states they are in from the
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ],
    columns=["State","RegionName"]  )'''
    df = pd.read_csv('university_towns.txt', dtype=str, delimiter='|', header=None)

    import re
    df = df.apply(lambda x: re.sub(' \(.*', '', x[0]), axis=1)  # x is Series data type

    index = 0
    state_region = []
    current_state = ''
    for i in range(len(df)):
        name = df.iloc[i]
        # get the state name, and skip to next line
        if "[edit]" in name:
            current_state = name.replace('[edit]', '')
            continue
        # construct one line of data frame
        state_region.append(pd.Series({'State': current_state, 'RegionName': name}))
        index += 1
    # convert
    df_final = pd.DataFrame(state_region, index=range(len(state_region)), columns=['State', 'RegionName'])
    return df_final

In [4]:
def get_gdp():
    import pandas as pd
    # Load data
    df = pd.read_excel("gdplev.xls", skiprows=5)
    df = df.iloc[2:, 4:-1]
    df.index -= 2  # 使index从0开始
    df.columns = ['Year_Quarter', 'Current', '2009']
    return df


def get_recession_start():
    '''Returns the year and quarter of the recession start time as a
    string value in a format such as 2005q3'''
    df = get_gdp()
    # only look at GDP data from the first quarter of 2000 onward
    _2000q1 = df[df['Year_Quarter'] == '2000q1'].index[0]

    gdp = df['Current']
    for i in range(_2000q1, len(df)):
        if (gdp[i] < gdp[i - 1]) and (gdp[i - 1] < gdp[i - 2]):
            # print df.loc[i, 'Year_Quarter']
            # get back to 2 Q ago...
            start = df.loc[i - 2, 'Year_Quarter']
            break
    return start

In [5]:
def get_recession_end():
    df = get_gdp()
    # only look at GDP data from the first quarter of 2000 onward
    start = get_recession_start()
    start = df[df['Year_Quarter'] == start].index[0] + 2

    gdp = df['Current']
    for i in range(start, len(df)):
        if (gdp[i] > gdp[i - 1]) and (gdp[i - 1] > gdp[i - 2]):
            end = df.loc[i, 'Year_Quarter']
            break
    return end

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a
    string value in a format such as 2005q3'''
    df = get_gdp()
    df = df.set_index('Year_Quarter')

    start = get_recession_start()
    end = get_recession_end()
    bottom = df.loc[start: end]['Current'].argmin()
    return bottom

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].

    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.

    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # Load data
    import pandas as pd
    df = pd.read_csv('City_Zhvi_AllHomes.csv')

    # replace to state name by mapping
    df['State'] = df['State'].apply(lambda x: states.get(x))

    print(df.columns)
    df['2016-09'] = (df['2016-07'] + df['2016-08']).mean()
    years = df.columns[6:]  # ignore region name, state, etc.
    years = [int(x.replace('-', '')) for x in years]  # convert into a number
    _2000 = [x for x in years if x > 200000]  # filter only > 200001
    _2000 = [str(x // 100) + '-' + str(x % 100).zfill(2) for x in _2000 if x > 200000]  # convert to df column name

    df_final = pd.DataFrame()
    df_final['State'] = df['State']
    df_final['RegionName'] = df['RegionName']
    for i in range(0, len(_2000), 3):
        y = _2000[i].split('-')  # 2000-01 -> 2000, 01
        new_column = y[0] + 'q' + str((int(y[1]) // 3) + 1) # (int('01')//3) + 1 = 1//3 + 1 = 1
        df_final[new_column] = df[[_2000[i], _2000[i + 1], _2000[i + 2]]].mean(axis=1)

    df_final.set_index(['State', 'RegionName'], inplace=True)
    return df_final

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values,
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence.

    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    # Load data
    uni_towns = get_list_of_university_towns()
    prices = convert_housing_data_to_quarters()

    # get related years
    start = get_recession_start()
    years = prices.columns
    before = years[years.tolist().index(start) - 1]
    bottom = get_recession_bottom()
    print('year quarter start recession: {}, before recession: {}, recession bottom: {}'.format(start, before, bottom))
    # year quarter start recession: 2008q3, before recession: 2008q2, recession bottom: 2009q2

    # according to the Hypothesis description
    prices['PriceRatio'] = prices[before] / prices[bottom]

    # tell which group
    uni_towns['UniversityInd'] = 1
    uni_towns.set_index(['State', 'RegionName'], inplace=True)
    import pandas as pd
    prices = pd.merge(prices, uni_towns, how='left', left_index=True, right_index=True)
    prices['UniversityInd'] = prices['UniversityInd'].fillna(0)

    # get 2 t-test data set
    df_ut = prices[prices['UniversityInd'] == 1]
    df_ut = df_ut.dropna()
    # print df_ut['PriceRatio']

    df_nut = prices[prices['UniversityInd'] == 0]
    df_nut = df_nut.dropna()
    # print df_nut['PriceRatio']

    from scipy.stats import ttest_ind
    t_test_result = ttest_ind(df_ut['PriceRatio'], df_nut['PriceRatio'])
    p = t_test_result[1]
    different = p < 0.01
    if df_ut['PriceRatio'].mean() <= df_nut['PriceRatio'].mean():
        better = 'university town'
    else:
        better = 'non-university town'
    return different, p, better
